In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
from virny.datasets import DiabetesDataset2019
from virny.utils.custom_initializers import create_config_obj
from virny.custom_classes.metrics_composer import MetricsComposer
from virny.preprocessing.basic_preprocessing import preprocess_dataset
from virny.user_interfaces.multiple_models_api import compute_metrics_with_config

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from copy import deepcopy
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from openbox import Optimizer
from openbox.utils.config_space import (UniformFloatHyperparameter, Constant, UniformIntegerHyperparameter,
                                        ConfigurationSpace, Configuration)

## Load dataset

In [4]:
DATASET_SPLIT_SEED = 100
MODELS_TUNING_SEED = 100
TEST_SET_FRACTION = 0.2

In [5]:
config_yaml_path = os.path.join(os.getcwd(), 'experiment_config.yaml')
config_yaml_content = """
dataset_name: Diabetes
bootstrap_fraction: 0.8
random_state: 42
n_estimators: 2
sensitive_attributes_dct: {'Gender': 'Female',  'Age': 'less than 40', 'Gender & Age': None}
"""

with open(config_yaml_path, 'w', encoding='utf-8') as f:
    f.write(config_yaml_content)

metric_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

In [6]:
data_loader = DiabetesDataset2019(with_nulls=False)
data_loader.full_df.head()

,Age,Gender,Family_Diabetes,highBP,PhysicallyActive,BMI,Smoking,Alcohol,Sleep,SoundSleep,RegularMedicine,JunkFood,Stress,BPLevel,Pregnancies,Pdiabetes,UriationFreq,Diabetic
0,50-59,Male,no,yes,one hr or more,39.0,no,no,8,6,no,occasionally,sometimes,high,0.0,0,not much,0
1,50-59,Male,no,yes,less than half an hr,28.0,no,no,8,6,yes,very often,sometimes,normal,0.0,0,not much,0
2,40-49,Male,no,no,one hr or more,24.0,no,no,6,6,no,occasionally,sometimes,normal,0.0,0,not much,0
3,50-59,Male,no,no,one hr or more,23.0,no,no,8,6,no,occasionally,sometimes,normal,0.0,0,not much,0
4,40-49,Male,no,no,less than half an hr,27.0,no,no,8,8,no,occasionally,sometimes,normal,0.0,0,not much,0


In [7]:
data_loader.full_df.shape

(905, 18)

## Data preparation

In [8]:
column_transformer = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(categories='auto', handle_unknown='ignore', sparse=False), data_loader.categorical_columns),
        ('num', StandardScaler(), data_loader.numerical_columns),
    ],
    verbose_feature_names_out=False
)

In [9]:
base_flow_dataset = preprocess_dataset(data_loader=data_loader,
                                       column_transformer=column_transformer,
                                       sensitive_attributes_dct=metric_computation_config.sensitive_attributes_dct,
                                       test_set_fraction=TEST_SET_FRACTION,
                                       dataset_split_seed=DATASET_SPLIT_SEED)

In [10]:
x_train, x_val, y_train, y_val = train_test_split(base_flow_dataset.X_train_val, base_flow_dataset.y_train_val,
                                                  test_size=TEST_SET_FRACTION, random_state=1)
base_flow_dataset_for_hpo = deepcopy(base_flow_dataset)
base_flow_dataset_for_hpo.X_train_val = x_train
base_flow_dataset_for_hpo.y_train_val = y_train
base_flow_dataset_for_hpo.X_test = x_val
base_flow_dataset_for_hpo.y_test = y_val

## HPO with OpenBox

In [11]:
# def get_lightgbm_config_space():
#     cs = ConfigurationSpace()
#     n_estimators = UniformIntegerHyperparameter("n_estimators", 100, 1000, default_value=500, q=50)
#     num_leaves = UniformIntegerHyperparameter("num_leaves", 31, 2047, default_value=128)
#     max_depth = Constant('max_depth', 15)
#     learning_rate = UniformFloatHyperparameter("learning_rate", 1e-3, 0.3, default_value=0.1, log=True)
#     min_child_samples = UniformIntegerHyperparameter("min_child_samples", 5, 30, default_value=20)
#     subsample = UniformFloatHyperparameter("subsample", 0.7, 1, default_value=1, q=0.1)
#     colsample_bytree = UniformFloatHyperparameter("colsample_bytree", 0.7, 1, default_value=1, q=0.1)
#     cs.add_hyperparameters([n_estimators, num_leaves, max_depth, learning_rate, min_child_samples, subsample,
#                             colsample_bytree])
#     return cs


def get_random_forest_config_space():
    cs = ConfigurationSpace()
    n_estimators = UniformIntegerHyperparameter("n_estimators", 50, 500, default_value=250, q=50)
    max_depth = UniformIntegerHyperparameter("max_depth", 10, 100, default_value=None, q=10)
    min_samples_split = UniformIntegerHyperparameter("min_samples_split", 5, 10, default_value=5, q=1)
    min_samples_leaf = UniformIntegerHyperparameter("min_samples_leaf", 1, 4, default_value=2, q=1)
    cs.add_hyperparameters([n_estimators, max_depth, min_samples_split, min_samples_leaf])
    
    return cs


def get_objective_function(base_flow_dataset, metric_computation_config):

    def cls_objective_function(config: Configuration):
        # convert Configuration to dict
        params = config.get_dictionary().copy()

        # model = LGBMClassifier(**params, verbose=-1)
        model = RandomForestClassifier(**params)
        models_config = {'RandomForestClassifier': model}

        metrics_dct = compute_metrics_with_config(base_flow_dataset, metric_computation_config, models_config,
                                                  save_results_dir_path=None,
                                                  notebook_logs_stdout=False)
        model_overall_metrics_df = metrics_dct[list(models_config.keys())[0]]
        
        metrics_composer = MetricsComposer(metrics_dct, metric_computation_config.sensitive_attributes_dct)
        models_composed_metrics_df = metrics_composer.compose_metrics()
        models_composed_metrics_df = models_composed_metrics_df[models_composed_metrics_df.Model_Name == 'RandomForestClassifier']

        # OpenBox minimizes the objective
        accuracy_loss = 1 - model_overall_metrics_df[model_overall_metrics_df.Metric == 'F1']['overall'].values[0]
        fairness_loss = abs(models_composed_metrics_df[models_composed_metrics_df.Metric == 'Equalized_Odds_TPR']['Gender&Age'].values[0])
        
        # Return result dictionary
        result = dict(objectives=[accuracy_loss, fairness_loss])
        return result

    return cls_objective_function

In [12]:
# Get config_space and objective_function
config_space = get_random_forest_config_space()
objective_function = get_objective_function(base_flow_dataset=base_flow_dataset_for_hpo,
                                            metric_computation_config=metric_computation_config)

# Run
ref_point = [0.2, 0.2] # [1 - min acceptable accuracy, min acceptable fairness]
opt = Optimizer(
    objective_function,
    config_space,
    num_objectives=2,
    num_constraints=0,
    max_runs=20,
    surrogate_type='auto',
    acq_type='auto',
    acq_optimizer_type='auto',
    initial_runs=9,
    init_strategy='sobol',
    ref_point=ref_point,
    task_id='mo',
    random_state=1,
    visualization='advanced',  # or 'basic'. For 'advanced', run 'pip install "openbox[extra]"' first
    auto_open_html=True,  # open the visualization page in your browser automatically
)
history = opt.run()
print(history)

[2024-10-24 00:54:36,426][mo][INFO][color_logger.py:203] Logfile: /Users/denys_herasymuk/Research/NYU/VirnyFlow_Project/Code/new-virny-flow/tests/logs/mo_2024-10-24-00-54-36-425739.log
[2024-10-24 00:54:36,436][mo][INFO][generic_advisor.py:227] [BO auto selection]  surrogate_type: gp. acq_type: ehvi. acq_optimizer_type: random_scipy.
[2024-10-24 00:54:36,841][mo][INFO][html_visualizer.py:462] Please open the html file to view visualization result: file:///Users/denys_herasymuk/Research/NYU/VirnyFlow_Project/Code/new-virny-flow/tests/logs/history/mo/mo_2024-10-24-00-54-36-839730.html
  0%|          | 0/20 [00:00<?, ?it/s]


Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap: 100%|██████████| 2/2 [00:05<00:00,  2.97s/it]

Analyze multiple models: 100%|██████████| 1/1 [00:06<00:00,  6.43s/it]

[2024-10-24 00:54:43,554][mo][INFO][generic_smbo.py:310] Iter 1, objectives: [0.16049382716049387, 0.10810810810810811].
[2024-10-24 00:54:43,557][mo][INFO][history.py:738] Computing all hypervolumes...
  5%|▌         | 1/20 [00:06<02:02,  6.46s/it]



Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap: 100%|██████████| 2/2 [00:00<00:00, 13.41it/s]

Analyze multiple models: 100%|██████████| 1/1 [00:00<00:00,  4.66it/s]

[2024-10-24 00:54:43,782][mo][INFO][generic_smbo.py:310] Iter 2, objectives: [0.15000000000000002, 0.10810810810810811].
[2024-10-24 00:54:43,784][mo][INFO][history.py:738] Computing all hypervolumes...
 10%|█         | 2/20 [00:06<00:50,  2.79s/it]

...



Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap: 100%|██████████| 2/2 [00:00<00:00,  5.47it/s]

Analyze multiple models: 100%|██████████| 1/1 [00:00<00:00,  2.34it/s]

[2024-10-24 00:55:06,594][mo][INFO][generic_smbo.py:310] Iter 20, objectives: [0.1428571428571429, 0.05405405405405406].
[2024-10-24 00:55:06,596][mo][INFO][history.py:738] Computing all hypervolumes...
[2024-10-24 00:55:06,624][mo][INFO][html_visualizer.py:310] Verify surrogate model...



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000009 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 23
[LightGBM] [Info] Number of data points in the train set: 20, number of used features: 4
[LightGBM] [Info] Start training from score 0.156251
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

[2024-10-24 00:55:07,412][mo][INFO][html_visualizer.py:96] Please open the html file to view visualization result: file:///Users/denys_herasymuk/Research/NYU/VirnyFlow_Project/Code/new-virny-flow/tests/logs/history/mo/mo_2024-10-24-00-54-36-839730.html
100%|██████████| 20/20 [00:30<00:00,  1.52s/it]

+-------------------+---------------+
| Parameters        | Optimal Value |
+-------------------+---------------+
| max_depth         | 40            |
| min_samples_leaf  | 2             |
| min_samples_split | 10            |
| n_estimators      | 350           |
+-------------------+---------------+
| Objective 1       | 0.142857      |
| Objective 2       | 0.054054      |
+-------------------+---------------+
| Num Trials        | 20            |
+-------------------+---------------+
